In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import math 
import itertools as it

In [8]:
'''
    Import all data
'''

data = np.zeros((48384, 144))

for i in range(1,25):

    # Format the number with leading zeros
    number = f"{i:02d}"
    
    # Path of file
    base_path = f"data/X{number}/X{number}"

    # Load file
    temp = np.loadtxt(base_path)

    # Keep first 144 entries from each line 
    temp = temp[:, :144]

    # Append data    
    data[(i-1)*2016:2016*i, :] = temp

### Stationarity, Seasonality, etc. 

### Correlation Profile

In [11]:
'''
    Matrix normlaize data
'''

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, 1)) 

def normalize_matrix(scaler, dataset):

    dataset_norm = np.zeros(dataset.shape)

    for i in range(len(dataset)):
        row = np.reshape(dataset[i, :], (144,1))
        row = scaler.fit_transform(row)

        dataset_norm[i, :] = np.reshape(row, (144,))

    return dataset_norm

dataset_norm = normalize_matrix(scaler, data)


In [16]:
'''
    Compute inter-flow correlations...fix this
'''

od_pairs_1 = it.product(range(0,12), repeat = 2)
od_pairs_2 = it.product(range(0,12), repeat = 2)

inter_flow = []

for (s,d) in od_pairs_1: 
    for(s_1, d_1) in od_pairs_2:

        # Check that pairs aren't the same
        if (s_1 != s) & (d_1 != d):

            od_num_1  = s*12+d
            od_num_2  = s_1*12+d_1

            pair_1 = dataset_norm[:, od_num_1]
            pair_2 = dataset_norm[:, od_num_2]

            # get variance
            var_1 = np.var(pair_1)
            var_2 = np.var(pair_2)

            # get correlation
            cov = np.cov(pair_1, pair_2)[0][1]

            # compute correlation coefficient
            rho = cov/(var_1*var_2)

            # save value 
            inter_flow.append(rho)

inter_flow = np.array(inter_flow)

In [37]:
s, d = (1,10)
s_1, d_1, = (1,10)

od_num_1  = s*12+d
od_num_2  = s_1*12+d_1

pair_1 = data[:, od_num_1]
pair_2 = data[:, od_num_2]

# get variance
var_1 = np.var(pair_1)
var_2 = np.var(pair_2)

# get correlation
cov = np.cov(pair_1, pair_2)
cov




array([[6.47126749e+10, 6.47126749e+10],
       [6.47126749e+10, 6.47126749e+10]])

In [36]:
from scipy.stats import pearsonr

# Compute Pearson correlation coefficient and p-value
corr_coefficient, p_value = pearsonr(pair_1.reshape(-1), pair_2.reshape(-1))

print("Correlation Coefficient:", corr_coefficient)
print("P-value:", p_value)

Correlation Coefficient: nan
P-value: nan


C:\Users\marth\AppData\Local\Temp\ipykernel_29032\1140896686.py:4: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  corr_coefficient, p_value = pearsonr(pair_1.reshape(-1), pair_2.reshape(-1))
